In [598]:
from os import listdir, path
from os.path import isfile, join
import json
from lxml import html
from bs4 import BeautifulSoup
import string

import requests,json
from requests import get
from requests.auth import HTTPBasicAuth, HTTPDigestAuth
import urllib
import itertools
from collections import defaultdict
from collections import Counter

import numpy

# pytorch
from transformers import AutoTokenizer, AutoModelForMaskedLM

from nltk.tokenize import WhitespaceTokenizer
from sklearn.metrics import f1_score

In [225]:
def find_whitespace(st):
    for index, character in enumerate(st):
        if character in string.whitespace:
            yield index

In [83]:
def get_combinations(l):
    yield from itertools.product(l, l)

In [84]:
project_name = 'RoB_preliminary_annotation'
owner = 'anjDhr'
users = ['rahel-caliesch', 'roger-annotation', 'martin-annotation', 'katia-giacomino', 'simone-annotation']
all_user_combinations = list(itertools.combinations(users, 2)) 
all_user_combinations = get_combinations(users)
all_user_combinations = list( all_user_combinations )

In [85]:
'''
exact_v1 = Stringent inter-annotator agreement
overlapping_v1 = Relaxed inter-annotator agreement
'''
metrics = ['exact_v1', 'overlapping_v1'] 

In [86]:
# Get all the entities
all_annotations_data = 'https://www.tagtog.net/-api/metrics/v0/search_stats?project=RoB_preliminary_annotation&owner=anjDhr&search=*'
all_annotations_data_response = get(all_annotations_data, auth=('anjDhr', '9J@NiScMhUy9LbR'))
print('The response for query is: ', all_annotations_data_response)
all_annotations_data_response = json.loads(all_annotations_data_response.text)

The response for query is:  <Response [200]>


In [87]:
# Get the IAA metrics
# IAA_hardcoded = 'https://www.tagtog.net/-api/metrics/v0/iaa?project=RoB_preliminary_annotation&owner=anjDhr&member1=rahel-caliesch&member2=roger-annotation&anntaskId=e_109&metric=exact_v1'
IAA = 'https://www.tagtog.net/-api/metrics/v0/iaa?'

In [88]:
global_annot = dict()
user_agreement = dict() # what users have constant high agreement (relaxed agreement) between entities
entity2label = dict()

for eachEntry in all_annotations_data_response:
    
    if 'e_' in eachEntry:

        entry_name =  all_annotations_data_response[eachEntry]['name']
        entity_IAA = {}
        
        entity2label[eachEntry] = entry_name

        for eachUserPair in all_user_combinations:

            member1 = eachUserPair[0]
            member2 = eachUserPair[1]

            if member1 != member2:

                params = (('project',project_name),('owner', owner), ('member1', member1), ('member2', member2), ('anntaskId', eachEntry), ('metric', metrics[1])) # metrics[1] = overlapping_v1
                parameters = urllib.parse.urlencode(params)
                entire_command = IAA + parameters
                response = get(entire_command, auth=('anjDhr', '9J@NiScMhUy9LbR'))
                my_json_data = json.loads(response.text)

                if eachUserPair not in entity_IAA and tuple(reversed(eachUserPair)) not in entity_IAA:
                    entity_IAA[eachUserPair] = my_json_data['f1']
        global_annot[entry_name] = entity_IAA

In [451]:
entity2label.keys()

dict_keys(['e_178', 'e_109', 'e_205', 'e_138', 'e_121', 'e_199', 'e_116', 'e_127', 'e_160', 'e_156', 'e_171', 'e_195', 'e_167', 'e_132', 'e_184', 'e_142', 'e_164', 'e_152', 'e_163', 'e_174', 'e_188', 'e_210', 'e_190', 'e_166', 'e_208', 'e_115', 'e_133', 'e_214', 'e_122', 'e_177', 'e_111', 'e_137', 'e_147', 'e_146', 'e_158', 'e_155', 'e_126', 'e_203', 'e_113', 'e_194', 'e_187', 'e_143', 'e_151', 'e_198', 'e_183', 'e_140', 'e_110', 'e_204', 'e_169', 'e_119', 'e_216', 'e_211', 'e_200', 'e_215', 'e_125', 'e_202', 'e_136', 'e_159', 'e_173', 'e_154', 'e_176', 'e_114', 'e_207', 'e_193', 'e_162', 'e_123', 'e_165', 'e_213', 'e_186', 'e_118', 'e_157', 'e_170', 'e_130', 'e_197', 'e_182', 'e_124', 'e_180', 'e_107', 'e_129', 'e_135', 'e_148', 'e_150', 'e_179', 'e_168', 'e_144', 'e_131', 'e_201', 'e_189', 'e_212', 'e_217', 'e_161', 'e_175', 'e_206', 'e_149', 'e_196', 'e_153', 'e_120', 'e_172', 'e_192', 'e_141', 'e_117', 'e_181', 'e_128', 'e_209', 'e_112', 'e_145', 'e_191', 'e_139', 'e_185', 'e_134']

In [92]:
entity2label.values()

dict_values(['4_1_Yes_Bad', '1_1_ProbablyYes_Good', '5_1_No_Bad', '2_4_Yes_Bad', '1_3_ProbablyNo_Good', '4_5_ProbablyYes_Bad', '1_2_ProbablyNo_Bad', '2_1_No_Information', '3_1_No_Bad', '2_7_ProbablyNo_Good', '3_3_ProbablyNo_Good', '4_4_No_Good', '3_2_No_Information', '2_2_No_Information', '4_2_ProbablyYes_Bad', '2_4_No_Information', '3_2_ProbablyYes_Good', '2_6_No_Information', '3_2_Yes_Good', '3_4_ProbablyYes_Bad', '4_3_Yes_Bad', '5_2_No_Good', '4_3_No_Good', '3_2_ProbablyNo_Bad', '5_2_Yes_Bad', '1_2_No_Bad', '2_3_Yes_Bad', '5_3_ProbablyYes_Bad', '1_3_No_Information', '3_4_No_Information', '1_1_ProbablyNo_Bad', '2_3_No_Information', '2_5_No_Information', '2_5_ProbablyNo_Bad', '3_1_Yes_Good', '2_7_No_Good', '2_1_ProbablyNo_Good', '5_1_Yes_Good', '1_2_Yes_Good', '4_4_ProbablyYes_Bad', '4_2_No_Information', '2_5_Yes_Good', '2_6_ProbablyNo_Bad', '4_5_Yes_Bad', '4_2_Yes_Bad', '2_4_No_Good', '1_1_No_Bad', '5_1_ProbablyYes_Good', '3_3_ProbablyYes_Bad', '1_3_ProbablyYes_Bad', '5_3_ProbablyNo_

In [104]:
# Get plain texts

In [93]:
plain_text = '/home/anjani/rob-annotation/data/RoB_preliminary_annotation/plain.html/pool'
member_annotations = '/home/anjani/rob-annotation/data/RoB_preliminary_annotation/ann.json/members/'

In [94]:
# list down all the annotators

projet_admin = ['anjDhr']

member_dir = listdir( member_annotations )

In [137]:
# Parse plain text files for the trial annotation project

plain_text_dict = dict()

plain_texts = [f for f in listdir(plain_text) if isfile(join(plain_text, f))]

for plaintext_file in plain_texts:
    
    plaintext_path = path.join(plain_text, plaintext_file)
    
    #with open(plaintext_path, "r") as f:
    #    page = f.read()
    page = open(plaintext_path, encoding="utf8")  
    
    soup = BeautifulSoup(page)

    #print(soup.head.title.text)
    trial_doc_number = str(soup.head.title.text).replace('.pdf', '')
    trial_doc_number = trial_doc_number.split('_')[-1]

    text_list = soup.find_all("pre")
    
    document_parts = {}
    for l in text_list:
        document_parts[ l.get('id') ] = l.text
    
    plain_text_dict[trial_doc_number] = document_parts
    #for t in text_list:
    #    print( t.get('id') )
    #    print( t.text )

In [96]:
# List member annotation files

member_annot_files = {}

for m, member in enumerate(member_dir):
    
    if member not in projet_admin:
        
        member_annot_dir = member_annotations + str(member) + '/'
        
        member_annot_dir = path.join(member_annotations, member, 'pool')
                
        annotation_files = [path.join(member_annot_dir, f) for f in listdir(member_annot_dir) if isfile(join(member_annot_dir, f))]
        
        annotation_dict = {}
        
        # generate a dictionary of document_number : annotations parsed
        for annotation_file in annotation_files:
            
            trial_doc_number = str(annotation_file).replace('.pdf.ann.json', '')
            trial_doc_number = trial_doc_number.split('_')[-1]
            
            with open(annotation_file, 'r') as af:
                data = json.load(af)
                #print(data['entities'])
                annotation_dict[trial_doc_number] = data['entities']
        
        
        member_annot_files[member] = annotation_dict

    #if m == 1:
    #    break

In [146]:
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [153]:
tk = WhitespaceTokenizer()

In [453]:
def flatenAnnotations(annotator1, annotator2, annotations_all):
    
    annot1 = annotations_all[annotator1]
    annot2 = annotations_all[annotator2]
    annot1_flattened = []
    annot2_flattened = []
    
    docs = list(annot1.keys())
    
    for doc in docs:
        #print('Document number: ', doc)
        doc_parts = list(annot1[doc].keys())
        
        for doc_part in doc_parts:
            #print( doc_part )
            if doc_part in annot1[doc] and doc_part in annot2[doc]:
                annot1_flattened.extend( annot1[doc][doc_part]['labels'] )
                annot2_flattened.extend( annot2[doc][doc_part]['labels'] )
                
            elif doc_part in annot1[doc] and doc_part not in annot2[doc]:
                annot1_flattened.extend( annot1[doc][doc_part]['labels'] )
                
                temp2 = [[0]] * len( annot1[doc][doc_part]['labels'] )
                annot2_flattened.extend( temp2 )
                
            elif doc_part not in annot1[doc] and doc_part in annot2[doc]:
                temp1 = [[0]] * len( annot2[doc][doc_part]['labels'] )
                annot1_flattened.extend( temp1 )
                
                annot2_flattened.extend( annot2[doc][doc_part]['labels'] )
                
    assert len( annot1_flattened ) == len( annot2_flattened )
    
    return annot1_flattened, annot2_flattened

In [322]:
def parseAnnotations(member_annotations):

    annotation_docs = dict()

    for k,v in member_annotations.items():

        annotation_dict = dict()

        for counter, a in enumerate(v):

            document_part = a['part']
            offset_start = a['offsets'][0]['start']
            offset_end = offset_start + len(a['offsets'][0]['text'])
            entity_text = a['offsets'][0]['text']
            document = plain_text_dict[k][document_part]

            document_entity_match = document[offset_start:offset_end]    
            document_entity_match_label = a['classId']
            assert len(entity_text) == len(document_entity_match)

            match_label_list = len( entity_text ) * [document_entity_match_label]

            document_char_labels = [0] * len(document)
            document_char_labels[offset_start:offset_end] = match_label_list

            if document_part not in annotation_dict:
                annotation_dict[document_part] = [ document_char_labels ]
            else:
                annotation_dict[document_part].append( document_char_labels )

        annotation_docs[k] = annotation_dict
        
    return annotation_docs

In [370]:
def char2tokAnnot(char_annotations):
    
    token_annotations = {}
    
    for k_a, v_a in char_annotations.items():
        #print(k_a) # document number
        
        token_annotations[k_a] = {}

        for k_a_, v_a_ in v_a.items(): # convert char annot to tok annot
            #print(k_a_) # document part

            text =  plain_text_dict[k_a][k_a_]
            white_space_spans = list(WhitespaceTokenizer().span_tokenize(plain_text_dict[k_a][k_a_]))
            tokens = [ text[ ws[0] : ws[1] ] for ws in  white_space_spans ]
            
            for v_a_i in v_a_:
            
                labels = [ list( set(v_a_i[ ws[0] : ws[1] ]) ) for ws in  white_space_spans ]
                #labels_clean = [ list(filter(lambda num: num != 0, l))[0] if len( l ) > 1 else l[0] for l in labels ]
                labels_clean = [ list(filter(lambda num: num != 0, l)) if len( l ) > 1 else l for l in labels ]
                #print('Clean labels: ' , labels_clean)
            
                assert len(v_a_i) == len(plain_text_dict[k_a][k_a_])                
                assert len( tokens ) == len( labels )
            
                if k_a_ not in token_annotations[k_a]:
                    token_annotations[k_a][k_a_] = {}
                    token_annotations[k_a][k_a_]['tokens'] = tokens
                    #token_annotations[k_a][k_a_]['labels'] = [ labels_clean ]
                    token_annotations[k_a][k_a_]['labels'] = labels_clean
                elif k_a_ in token_annotations[k_a]:
                    old_labels = token_annotations[k_a][k_a_]['labels']
                    new_labels = [ list(set(old_labels[n] + l)) for n, l in enumerate(labels_clean)]
                    
                    #new_labels = [ old_labels[n].append( l ) for n, l in enumerate(labels_clean) ]
                    #print( new_labels )
                    
                    assert len( old_labels ) == len( new_labels )
                    
                    token_annotations[k_a][k_a_]['labels'] = new_labels
            #break
                
    return token_annotations

In [373]:
annotations = dict()

for k,v in member_annot_files.items():

    if len( list(v.keys()) ) == 10: # restricts it to those who completed annotations
        print(k)
        char_annotations = parseAnnotations(v)
        tok_annotations = char2tokAnnot(char_annotations)
        annotations[k] = tok_annotations

roger-annotation
martin-annotation
katia-giacomino


In [609]:
# Calculate IAA

def calculate_exact_IAA(annotator1, annotator2, annotations_all):
    
    # Calculate different levels of inter-annotator agreement between the two annotators
    
    print( 'Calculating IAA between ', annotator1, ' and ', annotator2 )
  
    annot1_flat, annot2_flat = flatenAnnotations(annotator1, annotator2, annotations_all)
    
    # Completely flat lists
    a1_flat = [x for xs in annot1_flat for x in xs]
    a2_flat = [x for xs in annot2_flat for x in xs]
    
    entities = list( set( a1_flat + a2_flat ) )
    entities.remove(0)
        
    # Get the annotator labels 
    annotator1_labels = dict.fromkeys( entities, len(annot1_flat) * [0] )
    annotator2_labels = dict.fromkeys( entities, len(annot1_flat) * [0] )
    
    label1 = []
    label2 = []
    
    for i, (a1, a2) in enumerate( zip(annot1_flat, annot2_flat) ):
        
        temp1_ent_dict = dict.fromkeys( entities, 0 )
        temp2_ent_dict = dict.fromkeys( entities, 0 )
                       
        for a1_i in a1:
            if a1_i != 0:
                temp1_ent_dict[a1_i] = 1
                
        for a2_i in a2:
            if a2_i != 0:
                temp2_ent_dict[a2_i] = 1
                
        temp1_ent_list = []
        temp2_ent_list = []
        
        for e in entities:
            temp1_ent_list.append( temp1_ent_dict[e] )
            temp2_ent_list.append( temp2_ent_dict[e] )
            
        #Append to the main list
        if sum( temp1_ent_list ) > 0 or sum( temp2_ent_list ) > 0:
            label1.append( temp1_ent_list )
            label2.append( temp2_ent_list )
        
    f1_iaa = f1_score(label1, label2, average=None)
    
    for e, f1 in zip(entities, f1_iaa):
        if f1 > 0:
            print( entity2label[e], ' : ', f1*100 )

In [610]:
calculate_IAA_F1('roger-annotation', 'katia-giacomino', annotations)

Calculating IAA between  roger-annotation  and  katia-giacomino
1_2_ProbablyYes_Good  :  18.29652996845426
1_3_No_Information  :  75.3694581280788
3_1_ProbablyYes_Good  :  3.324099722991689
1_3_ProbablyNo_Good  :  55.420412480169226
2_6_Yes_Good  :  33.54037267080745
4_1_ProbablyNo_Good  :  0.9508716323296353
3_1_Yes_Good  :  98.06201550387598
4_2_ProbablyNo_Good  :  1.8316308559351884
1_1_Yes_Good  :  16.09195402298851
4_1_No_Good  :  12.334801762114537
1_1_ProbablyYes_Good  :  4.109589041095891
1_2_No_Information  :  60.67415730337078


In [645]:
def calculate_judgeless_IAA(annotator1, annotator2, annotations_all):
    
    # Calculate different levels of inter-annotator agreement between the two annotators
    
    print( 'Calculating IAA between ', annotator1, ' and ', annotator2 )
  
    annot1_flat, annot2_flat = flatenAnnotations(annotator1, annotator2, annotations_all)
    
    # Completely flat lists
    a1_flat = [entity2label[x][0:3] if x != 0 else x for xs in annot1_flat for x in xs]
    a2_flat = [entity2label[x][0:3] if x != 0 else x for xs in annot2_flat for x in xs]

    entities = list( set( a1_flat + a2_flat ) )
    entities.remove(0)

    label1 = []
    label2 = []
    
    for i, (a1, a2) in enumerate( zip(annot1_flat, annot2_flat) ):
        
        temp1_ent_dict = dict.fromkeys( entities, 0 )
        temp2_ent_dict = dict.fromkeys( entities, 0 )
        
        for a1_i in a1:
            if a1_i != 0:
                relax1_ent = entity2label[a1_i][0:3]
                if relax1_ent in temp1_ent_dict:
                    temp1_ent_dict[relax1_ent] = 1
                
        for a2_i in a2:
            if a2_i != 0:
                relax2_ent = entity2label[a2_i][0:3]
                if relax2_ent in temp2_ent_dict:
                    temp2_ent_dict[relax2_ent] = 1
                    
        temp1_ent_list = []
        temp2_ent_list = []
        
        for e in entities:
            temp1_ent_list.append( temp1_ent_dict[e] )
            temp2_ent_list.append( temp2_ent_dict[e] )
            
        #Append to the main list
        if sum( temp1_ent_list ) > 0 or sum( temp2_ent_list ) > 0:
            label1.append( temp1_ent_list )
            label2.append( temp2_ent_list )
            

    f1_iaa = f1_score(label1, label2, average=None)
    
    for e, f1 in zip(entities, f1_iaa):
        if f1 > 0:
            print( e, ' : ', f1*100 )

In [646]:
calculate_judgeless_IAA('roger-annotation', 'katia-giacomino', annotations)

Calculating IAA between  roger-annotation  and  katia-giacomino
2_1  :  0.966183574879227
3_2  :  1.3618677042801557
5_3  :  0.11560693641618495
4_2  :  1.7603249830737984
5_2  :  23.83177570093458
2_3  :  20.52980132450331
2_6  :  75.23629489603026
2_2  :  18.21036106750393
1_2  :  66.04434072345391
1_3  :  69.44444444444444
3_1  :  45.75707154742096
4_1  :  3.9627039627039626
1_1  :  23.05475504322767
4_3  :  7.569721115537847


In [647]:
calculate_judgeless_IAA('roger-annotation', 'martin-annotation', annotations)

Calculating IAA between  roger-annotation  and  martin-annotation
2_1  :  1.342281879194631
2_7  :  6.521739130434782
2_3  :  5.421686746987953
2_6  :  68.85245901639344
2_2  :  7.236842105263158
1_2  :  50.289017341040456
1_3  :  20.44801838024124
3_1  :  23.57274401473297
4_1  :  6.513872135102533
1_1  :  24.440298507462686
4_3  :  13.858267716535435


In [648]:
calculate_judgeless_IAA('katia-giacomino', 'martin-annotation', annotations)

Calculating IAA between  katia-giacomino  and  martin-annotation
2_1  :  9.068010075566752
3_2  :  3.263403263403264
5_3  :  0.32
2_6  :  63.89891696750903
1_2  :  50.68702290076336
1_3  :  31.536388140161726
3_1  :  43.35664335664335
4_1  :  25.60777957860616
1_1  :  56.95006747638326
3_3  :  16.3265306122449
4_3  :  10.502283105022832


In [514]:
# Calculate IAA

def calculate_IAA(annotator1, annotator2, annotations_all):
    
    # Calculate different levels of inter-annotator agreement between the two annotators
    
    print( 'Calculating IAA between ', annotator1, ' and ', annotator2 )
  
    annot1_flat, annot2_flat = flatenAnnotations(annotator1, annotator2, annotations_all)
    
    # Completely flat lists
    a1_flat = [x for xs in annot1_flat for x in xs]
    a2_flat = [x for xs in annot2_flat for x in xs]
    
    IAA_Po = dict.fromkeys( list(entity2label.keys()) )

    # Initialize valid annotated entities with 0
    annotated_entities = list(set( a1_flat + a2_flat ))
    for e in annotated_entities:
        if e in IAA_Po:
            IAA_Po[e] = 0
    
    for i, (a1, a2) in enumerate( zip( annot1_flat, annot2_flat ) ):
        
        if any(a1) != 0 and any(a2) != 0:
            
            unique = list(set(a1).intersection(a2))
            for count in unique:
                if count in IAA_Po:
                    IAA_Po[count] = IAA_Po[count] + 1
    
    uniq = set(a1_flat + a2_flat)
    E = 0  # expected agreement E (Pe)
    for item in uniq:
        cnt1 = a1_flat.count(item)
        cnt2 = a2_flat.count(item)
        count = ((cnt1 / len(a1_flat)) * (cnt2 / len(a2_flat)))
        E += count
        
    IAA = dict.fromkeys( list(entity2label.keys()), 0 )
    
    for k,v in IAA_Po.items():
        if v != 0 and v != None:
            
            iaa_agreement = round( (v - E) / (1 - E), 4 )
            
            print( k ,  iaa_agreement )